In [ ]:
""" Notebook for final model building evaluation"""

import os
import json
from pprint import pprint
from datetime import datetime
from collections import defaultdict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, Normalizer, scale, normalize
from sklearn.model_selection import (
    cross_val_predict,
    cross_val_score,
    cross_validate,
    KFold
)
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    make_scorer,
)
from sklearn.pipeline import make_pipeline

from dotenv import load_dotenv

from tc_data import TopCoder
from final_model_selection import (
    kfold_predict_validate_gradient_boosting,
    kfold_predict_validate_neural_network,
    train_gb_for_production
)

load_dotenv()
pd.set_option('display.max_rows', 800)

## Get the data object

`TopCoder` class contain the pre-processed data retrieved from data base.

In [ ]:
tc = TopCoder()

In [ ]:
X, y = tc.build_final_dataset('avg_score')

## Compare Gradient Boosting against NN

In [ ]:
hyper_param_dct = {
    'avg_score': {'gb': {}, 'nn': {}},
    'number_of_registration': {'gb': dict(tol=4), 'nn': dict(es_min_delta=4)},
    'sub_reg_ratio': {'gb': dict(loss='lad', tol=0.001, n_iter_no_change=10), 'nn': dict(es_min_delta=0.001)},
    'total_prize': {'gb': dict(loss='ls', tol=10, n_iter_no_change=5), 'nn': dict(es_min_delta=10)}
}

In [ ]:
res = {}
for target, param_dct in hyper_param_dct.items():
    X, y = tc.build_final_dataset(target)
    gb_y_pred, gb_cv_eval_df, gb_manual_score, gb_cv_fi = kfold_predict_validate_gradient_boosting(X, y, **param_dct['gb'])
    nn_y_pred, nn_cv_eval_df, nn_manual_score = kfold_predict_validate_neural_network(X, y, **param_dct['nn'])
    
    res[target] = dict(
        y=y,
        gb_y_pred=gb_y_pred,
        nn_y_pred=nn_y_pred,
        gb_cv_mean=gb_cv_eval_df.mean(),
        nn_cv_mean=nn_cv_eval_df.mean(),
        gb_manual=pd.Series(gb_manual_score),
        nn_manual=pd.Series(nn_manual_score)
    )

In [ ]:
multi_idx_dct = defaultdict(dict)
algo_name = {
    'gb': 'Gradient Boosting',
    'nn': 'Neural Network',
}
result_name = {
    'cv_mean': 'CV Mean',
    'manual': 'Full Set',
}

for target in res:
    for algo in ('gb', 'nn'):
        for result in ('cv_mean', 'manual'):
            for metric, score in res[target][f'{algo}_{result}'].iteritems():
                multi_idx_dct[(metric, result_name[result])][(target, algo_name[algo])] = score

In [ ]:
prediction_metrics = pd.DataFrame.from_dict(multi_idx_dct, orient='index').sort_index()
prediction_metrics

In [ ]:
avg_score_df = pd.read_json('./baseline/avg_score_result.json', orient='index')
number_of_registration_df = pd.read_json('./baseline/number_of_registration_result.json', orient='index')
sub_reg_ratio_df = pd.read_json('./baseline/sub_reg_ratio_result.json', orient='index')
total_prize_df = pd.read_json('./baseline/total_prize_result.json', orient='index')

In [ ]:
(res['avg_score']['gb_y_pred'].index == avg_score_df.index).all()

In [ ]:
res.keys()

In [ ]:
from scipy.stats import ttest_ind, ttest_rel

In [ ]:
ttest_ind(res['avg_score']['gb_y_pred'], avg_score_df['avg_score_pred'], equal_var=False).pvalue

In [ ]:
ttest_ind(res['number_of_registration']['gb_y_pred'], number_of_registration_df['number_of_registration_pred'], equal_var=False).pvalue


In [ ]:
ttest_ind(res['sub_reg_ratio']['gb_y_pred'], sub_reg_ratio_df['sub_reg_ratio_pred'], equal_var=False).pvalue

In [ ]:
ttest_ind(res['total_prize']['gb_y_pred'], total_prize_df['total_prize_pred'], equal_var=False).pvalue

In [ ]:
endpoint = [100, 300, 1, 2500]

with sns.axes_style('darkgrid'):
    fig, axes = plt.subplots(4, 2, figsize=(8, 16), dpi=200)
    for i, (target, predict_res) in enumerate(res.items()):
        for j, (algo, title) in enumerate(algo_name.items()):
            ax = axes[i, j]
            sns.scatterplot(
                x=predict_res[f'{algo}_y_pred'],
                y=predict_res['y'],
                s=20,
                alpha=0.5,
                linewidth=0.1,
                ax=ax,
            )
            sns.lineplot(
                x=[0, endpoint[i]],
                y=[0, endpoint[i]],
                color='red',
                alpha=0.75,
                ax=ax,
            )
            ax.set_title(f'{target.capitalize()} - {title}')
            ax.set_xlabel('Prediction')
            ax.set_ylabel('Ground Truth')
    fig.tight_layout()
    
#     fig.savefig('./result/img/pred_against_truth.png', dpi='figure')

## Inspect the feature importance of Gradient Boosting

In [ ]:
gb_param_dct = {
    'avg_score': {},
    'number_of_registration': dict(tol=4),
    'sub_reg_ratio': dict(loss='lad', tol=0.001, n_iter_no_change=10),
}

In [ ]:
gb_res = {}
for target, param in gb_param_dct.items():
    X, y = tc.build_final_dataset(target)
    y_pred, cv_eval_df, manual_score, cv_feature_importance = kfold_predict_validate_gradient_boosting(X, y, **param)

    gb_res[target] = dict(
        y=y,
        y_pred=y_pred,
        cv_mean=cv_eval_df.mean(),
        manual=pd.Series(manual_score),
        fea_importance = cv_feature_importance.mean()
    )
 

In [ ]:
midx_dct = defaultdict(dict)
for target, res_dct in gb_res.items():
    for result in 'cv_mean', 'manual':
        for metric, score in res_dct[result].items():
            midx_dct[target][(metric, result)] = score
        
res_df = pd.DataFrame.from_dict(midx_dct).sort_index()
res_df

            |approach|baseline|
            |.  |.  |.  ||.  |. |   |
    metric0|
    metric1|


In [ ]:
# res_df.to_json('result/final_models/extened_features_gradient_boosting_result.json')

In [ ]:
fea_imp_dct = {
    target: res_dct['fea_importance'] for target, res_dct in gb_res.items()
}

In [ ]:
palette = [f'#{color}' for color in'641220-6e1423-85182a-a11d33-a71e34-b21e35-bd1f36-c71f37-da1e37-e01e37'.split('-')]

In [ ]:
with sns.axes_style('darkgrid'):
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), dpi=200, sharex=True)
    for i, (target, feature_imp) in enumerate(fea_imp_dct.items()):
        ax = axes[i]

        sns.barplot(
            x=feature_imp.sort_values(ascending=False).head(10),
            y=feature_imp.sort_values(ascending=False).head(10).index,
            ax=ax,
            palette=palette[::-1]
        )
        ax.set_title('{} - Feature Importance'.format(' '.join([w.capitalize() for w in target.split('_')])))
        ax.set_xlim(0, 1)
        
        for p in ax.patches:
            x = p.get_width() + 0.01
            y = p.get_y() + 0.5 * p.get_height() + 0.002
            importance = round(p.get_width(), 3)
            ax.text(x, y, importance, va='center', fontdict={'fontsize': 8})
            
    fig.tight_layout()
#     fig.savefig('result/img/gradient_boosting_feature_importance.png', dpi='figure')

## Train model using full dataset

In [ ]:
gb_param_dct = {
    'avg_score': dict(),
    'number_of_registration': dict(tol=4),
    'sub_reg_ratio': dict(loss='lad', tol=0.001, n_iter_no_change=10),
}

In [ ]:
estimators_dct = {}
for target, param in gb_param_dct.items():
    print(f'\nTraining target: {target}...')
    X, y = tc.build_final_dataset(target)
    estimators_dct[target] = train_gb_for_production(X, y, target, **param)

In [ ]:
palette = [f'#{color}' for color in'641220-6e1423-85182a-a11d33-a71e34-b21e35-bd1f36-c71f37-da1e37-e01e37'.split('-')]
with sns.axes_style('darkgrid'):
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), dpi=200, sharex=True)
    for i, (target, pipeline) in enumerate(estimators_dct.items()):
        ax = axes[i]
        feature_imp = pd.Series(pipeline[f'{target}_gb_reg'].feature_importances_, index=X.columns)
        sns.barplot(
            x=feature_imp.sort_values(ascending=False).head(10),
            y=feature_imp.sort_values(ascending=False).head(10).index,
            ax=ax,
            palette=palette[::-1]
        )
        ax.set_title('{} - Feature Importance'.format(' '.join([w.capitalize() for w in target.split('_')])))
        ax.set_xlim(0, 1)
        
        for p in ax.patches:
            x = p.get_width() + 0.01
            y = p.get_y() + 0.5 * p.get_height() + 0.002
            importance = round(p.get_width(), 3)
            ax.text(x, y, importance, va='center', fontdict={'fontsize': 8})
            
    fig.tight_layout()
#     fig.savefig('result/img/gb_feature_imp_train_with_whole_data.png', dpi='figure')

In [ ]:
sample = X.sample(1).copy()

In [ ]:
sample_lst = []
for prize in range(100, 2500):
    s = sample.squeeze().copy()
    s['total_prize'] = prize
    sample_lst.append(s)

sample_df = pd.DataFrame.from_records(sample_lst)

In [ ]:
sample_df

In [ ]:
sample_df['total_prize']

In [ ]:
y_pred_dct = {}
for target, gb_reg in estimators_dct.items():
    y_pred_dct[target] = gb_reg.predict(sample_df.to_numpy())

In [ ]:
palette = [f'#{color}' for color in'641220-6e1423-85182a-a11d33-a71e34-b21e35-bd1f36-c71f37-da1e37-e01e37'.split('-')]
with sns.axes_style('darkgrid'):
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), dpi=200)
    for i, (target, y_pred) in enumerate(y_pred_dct.items()):
        ax = axes[i]

        sns.lineplot(x=sample_df['total_prize'], y=y_pred, color='orange', ax=ax)
        ax.set_xticks(list(range(100, 2600, 100)))
        ax.set_ylabel(target)
        ax.set_title(f'{target} prediction by posted prize')
    fig.tight_layout()

## Find sample case for background session

In [ ]:
cha_info = tc.get_filtered_challenge_info()

In [ ]:
cha_info

In [ ]:
expensive_cha = cha_info.loc[cha_info.total_prize >= 750]
cheap_cha = cha_info.loc[cha_info.total_prize <= 500]

while True:
    exp_sample = expensive_cha.sample(1).squeeze()
    cheap_sample = cheap_cha.sample(1).squeeze()
    
    score_diff = exp_sample['avg_score'] - cheap_sample['avg_score']
    reg_diff = exp_sample['number_of_registration'] - cheap_sample['number_of_registration']
    sub_diff = exp_sample['sub_reg_ratio'] - cheap_sample['sub_reg_ratio']
    dura_diff = exp_sample['challenge_duration'] - cheap_sample['challenge_duration']
    
    if (
        exp_sample['avg_score'] > 90 and 
        exp_sample['number_of_submitters'] > 1 and
        cheap_sample['number_of_submitters'] > 1 and
        score_diff >= 3 and 
        reg_diff > 0 and 
        sub_diff > 0 and 
        dura_diff == 0
    ):
        break

In [ ]:
pd.concat([exp_sample, cheap_sample], axis=1)

In [ ]:
pd.concat([exp_sample, cheap_sample], axis=1)

In [ ]:
pd.concat([exp_sample, cheap_sample], axis=1)

In [ ]:
pd.concat([exp_sample, cheap_sample], axis=1)

In [ ]:
pd.concat([exp_sample, cheap_sample], axis=1)

In [ ]:
pd.concat([exp_sample, cheap_sample], axis=1)